#### last checkpoint with code change: 8/15/2022

In [276]:
import os, sys
import geopandas as gpd
import pandas as pd
import rasterio as rio
import numpy as np
from shapely.geometry import Point
import skimage.graph as graph

In [277]:
import sys, os

In [278]:
sys.path.append('/home/wb411133/Code/gostrocks/src') # gostrocks is used for some basic raster operations (clip and standardize)
sys.path.append('/home/wb411133/Code/GOSTNets_Raster/src') # gostnets_raster has functions to work with friction surface
sys.path.append('/home/wb411133/Code/GOSTnets') # it also depends on gostnets for some reason
sys.path.append('/home/wb411133/Code/INFRA_SAP') # only used to save some raster results

In [279]:
# sys.path.append('/home/wb514197/Repos/gostrocks/src') # gostrocks is used for some basic raster operations (clip and standardize)
# sys.path.append('/home/wb514197/Repos/GOSTNets_Raster/src') # gostnets_raster has functions to work with friction surface
# sys.path.append('/home/wb514197/Repos/GOSTnets') # it also depends on gostnets for some reason
# sys.path.append('/home/wb514197/Repos/INFRA_SAP') # only used to save some raster results
# sys.path.append('/home/wb514197/Repos/HospitalAccessibility/src') # only used to save some raster results

In [280]:
import GOSTRocks.rasterMisc as rMisc
import GOSTNetsRaster.market_access as ma
from infrasap import aggregator

In [281]:
iso3 = 'LBR'

In [282]:
input_dir = "/home/public/Data/PROJECTS/Health" #
out_folder = os.path.join(input_dir, "output", iso3)
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

## Admin Boundaries

In [283]:
global_admin = '/home/public/Data/GLOBAL/ADMIN/g2015_0_simplified.shp'
adm0 = gpd.read_file(global_admin)
aoi = adm0.loc[adm0.ISO3166_1_==iso3]

In [284]:
global_admin2 = '/home/public/Data/GLOBAL/ADMIN/Admin2_Polys.shp'
adm2 = gpd.read_file(global_admin2)
adm2 = adm2.loc[adm2.ISO3==iso3].copy()
adm2 = adm2.to_crs("EPSG:4326")

### Destinations

## 1st round master lists

In [285]:
lbr_master = pd.read_excel(os.path.join(input_dir, "from_tashrik", "master lists", "Liberia.xlsx"))

In [286]:
lbr_master['status'] = lbr_master['Status'].apply(lambda x: x.lower())
lbr_master['hf_type'] = lbr_master['HF Type'].apply(lambda x: x.lower())
lbr = lbr_master.loc[lbr_master.Status!="non-functional"].copy()

In [287]:
lbr[['Lat']].isna().value_counts()

Lat  
False    627
True      72
dtype: int64

In [288]:
lbr = lbr.loc[~lbr.Lat.isna()].copy()
lbr[['hf_type']].value_counts()

hf_type      
clinic           538
health center     53
hospital          36
dtype: int64

In [289]:
geoms = [Point(xy) for xy in zip(lbr.Long, lbr.Lat)]
lbr_geo = gpd.GeoDataFrame(lbr, crs='EPSG:4326', geometry=geoms)

In [290]:
# test map round 1
lbr_geo.to_file(os.path.join(out_folder, "round_1_lbr_geo.tif"))

/tmp/ipykernel_22074/1498142498.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  lbr_geo.to_file(os.path.join(out_folder, "round_1_lbr_geo.tif"))


## new updated master lists

In [291]:
lbr_master = pd.read_csv(os.path.join(input_dir, "from_tashrik", "master lists", "Liberia MFL Adjusted.csv"), index_col=0)
lbr_geocoded = pd.read_csv(os.path.join(input_dir, "from_tashrik", "master lists", "Liberia MFL Adjusted Geocoded_Validated.csv"), index_col=0)
lbr = lbr_master.loc[lbr_master.functional=="functional"].copy()
lbr = lbr.loc[~(lbr.latitude.isna())].copy()
len(lbr), len(lbr_geocoded)

(817, 55)

In [292]:
lbr_geocoded.geocoding_method.value_counts()

Location from old Master List     43
District name and county query     8
Location from Health Sites IO      2
Fuzzy match to OSM by Tashrik      2
Name: geocoding_method, dtype: int64

In [293]:
lbr_geocoded = lbr_geocoded.loc[lbr_geocoded.geocoding_method!="District name and county query"].copy()
len(lbr), len(lbr_geocoded)
lbr = pd.concat([lbr, lbr_geocoded])
lbr.reset_index(drop=True, inplace=True)
lbr.columns
lbr.facilitytype.value_counts()

clinic           741
health center     88
hospital          24
health post        6
schoold based      4
SDP                1
Name: facilitytype, dtype: int64

In [294]:
geoms = [Point(xy) for xy in zip(lbr.longitude, lbr.latitude)]
lbr_geo = gpd.GeoDataFrame(lbr, crs='EPSG:4326', geometry=geoms)

In [295]:
#aoi['geometry'].plot()

In [296]:
#lbr_geo.plot()

In [297]:
lbr_geo = gpd.sjoin(lbr_geo, aoi, op='within')

/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [298]:
lbr_geo.facilitytype.value_counts()

clinic           735
health center     87
hospital          24
health post        6
schoold based      4
SDP                1
Name: facilitytype, dtype: int64

In [299]:
# test map round 2
lbr_geo.to_file(os.path.join(out_folder, "round_2_lbr_geo.geojson"))

In [300]:
lbr_geo.columns

Index(['admin_county', 'admin_district', 'today', 'latitude', 'longitude',
       '_getcoordinate_altitude', 'precision', 'facilitytype', 'ownership',
       'opendate', 'functional', 'offer_anc', 'offer_obstetric', 'offer_emonc',
       'offer_postpartum', 'offer_newborncare', 'offer_maternalnewboen',
       'offer_csection', 'offer_childimmune', 'offer_imci',
       'offer_essentialnutrition', 'offer_fpcouple', 'offer_fpcounsel',
       'offer_adolescentsrhr', 'offer_sexgender', 'offer_detectmanage',
       'offer_precenttreat', 'dup', 'update_date', 'geocoding_method',
       'facility', 'geometry', 'index_right', 'OBJECTID', 'STATUS',
       'DISP_AREA', 'ADM0_CODE', 'ADM0_NAME', 'STR0_YEAR', 'EXP0_YEAR',
       'Shape_Leng', 'ISO3166_1_', 'Country_Co', 'ISO3166_11', 'Country__1',
       'Shape_Le_1', 'Shape_Area'],
      dtype='object')

In [301]:
# counts of sub categories

In [302]:
len(lbr_geo.loc[lbr_geo['offer_anc'] == 1])

746

In [303]:
len(lbr_geo.loc[lbr_geo['offer_emonc'] == 1])

351

In [304]:
len(lbr_geo.loc[lbr_geo['offer_childimmune'] == 1])

674

In [305]:
len(lbr_geo.loc[lbr_geo['offer_essentialnutrition'] == 1])

437

In [306]:
len(lbr_geo.loc[lbr_geo['offer_adolescentsrhr'] == 1])

555

### Friction and Population

In [307]:
global_friction_surface = "/home/public/Data/GLOBAL/INFRA/FRICTION_2020/2020_motorized_friction_surface.geotiff"
global_population = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/ppp_2020_1km_Aggregated.tif"

In [308]:
inG = rio.open(global_friction_surface)

In [309]:
rMisc.clipRaster?

In [310]:
# Clip the travel raster to AOI
out_travel_surface = os.path.join(out_folder, "travel_surface.tif")
#rMisc.clipRaster(inG, aoi, out_travel_surface, bbox=False, buff=0.1)

In [311]:
aoi_w_buff = aoi.buffer(0.1)

/tmp/ipykernel_22074/1614152514.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoi_w_buff = aoi.buffer(0.1)


In [312]:
rMisc.clipRaster(inG, aoi_w_buff, out_travel_surface)

In [313]:
inP = rio.open(global_population)

In [314]:
# Clip the pop raster to AOI
out_pop = os.path.join(out_folder, "WP_2020_1km.tif")
rMisc.clipRaster(inP, aoi_w_buff, out_pop)

In [315]:
travel_surf = rio.open(out_travel_surface)
pop_surf = rio.open(out_pop)

In [316]:
#rMisc.standardizeInputRasters??

In [317]:
# standardize so that they have the same number of pixels and dimensions
out_pop_surface_std = os.path.join(out_folder, "WP_2020_1km_STD.tif")
#rMisc.standardizeInputRasters(pop_surf, travel_surf, inR1_outFile=out_pop_surface_std, resampling_type="nearest")
rMisc.standardizeInputRasters(pop_surf, travel_surf, inR1_outFile=out_pop_surface_std, data_type="C")

[array([[[-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
          -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
         [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
          -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
         [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
          -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
         ...,
         [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
          -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
         [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
          -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
         [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
          -3.4028235e+38, -3.4028235e+38, -3.4028235e+38]]], dtype=float32),
 {'driver': 'GTiff',
  'dtype': 'float32',
  'nodata': -3.4028234663852886e+38,
  'width': 521,
  'height': 528,
  'count': 1,
  'crs': CRS.from_epsg(4326),
  'transform': Affine(0.008333333333333333, 0.0, -11.599999999999994,
         

In [318]:
# create a data frame of all points
pop_surf = rio.open(out_pop_surface_std)
pop = pop_surf.read(1, masked=True)
indices = list(np.ndindex(pop.shape))
xys = [Point(pop_surf.xy(ind[0], ind[1])) for ind in indices]
res_df = pd.DataFrame({
    'spatial_index': indices, 
    'xy': xys, 
    'pop': pop.flatten()
})
res_df['pointid'] = res_df.index

/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [319]:
# create MCP object
inG_data = travel_surf.read(1) * 1000 # minutes to travel 1 meter, convert to km
# Correct no data values
inG_data[inG_data < 0] = 9999999999 # untraversable
# inG_data[inG_data < 0] = np.nan
mcp = graph.MCP_Geometric(inG_data)

Separate destinations

In [320]:
lbr_geo_filt = lbr_geo.loc[lbr_geo.intersects(aoi.unary_union)]

In [321]:
lbr_geo_filt.reset_index(inplace=True, drop=True)

In [322]:
len(lbr_geo), len(lbr_geo_filt)

(857, 857)

In [323]:
lbr_geo_filt[:3]

,admin_county,admin_district,today,latitude,longitude,_getcoordinate_altitude,precision,facilitytype,ownership,opendate,...,ADM0_NAME,STR0_YEAR,EXP0_YEAR,Shape_Leng,ISO3166_1_,Country_Co,ISO3166_11,Country__1,Shape_Le_1,Shape_Area
0,Montserrado,Somalia Drive,2021-08-07,6.331593,-10.743366,4.009424,4.983,clinic,Private (non-faith based),2013,...,Liberia,1000,3000,19.349878,LBR,LBR,LR,LR,19.349878,7.856866
1,Montserrado,Central Monrovia,2021-10-19,6.304997,-10.799709,2.900000,4.978,clinic,"Private, (faith based)",1988,...,Liberia,1000,3000,19.349878,LBR,LBR,LR,LR,19.349878,7.856866
2,Montserrado,St Paul River,2021-11-18,6.412980,-10.778231,-2.712283,4.933,clinic,Private (non-faith based),2018,...,Liberia,1000,3000,19.349878,LBR,LBR,LR,LR,19.349878,7.856866


In [324]:
lbr_geo_filt.facilitytype.value_counts()
#lbr_geo_filt.hf_type.value_counts()

clinic           735
health center     87
hospital          24
health post        6
schoold based      4
SDP                1
Name: facilitytype, dtype: int64

In [325]:
sub_category_list = ["offer_anc", 
"offer_emonc",
"offer_childimmune", 
"offer_essentialnutrition", 
"offer_adolescentsrhr"
]

In [326]:
sub_category_list

['offer_anc',
 'offer_emonc',
 'offer_childimmune',
 'offer_essentialnutrition',
 'offer_adolescentsrhr']

In [327]:
res_df.shape

(275088, 4)

In [328]:
# clean and prepare res_df
#res_df = res_df.loc[res_df['pop']>0].copy()
#res_df = res_df.loc[~(res_df['pop'].isna())].copy()
res_df.loc[:,'xy'] = res_df['xy'].apply(Point)
res_gdf = gpd.GeoDataFrame(res_df, geometry='xy', crs='EPSG:4326')
res_gdf.loc[:, 'geometry'] = res_gdf.loc[:, 'xy']

### test

In [329]:
print(len(lbr_geo_filt.loc[lbr_geo_filt['offer_anc']==1]))
res = ma.calculate_travel_time(travel_surf, mcp, lbr_geo_filt.loc[lbr_geo_filt['offer_anc']==1])[0].copy()
print(res.mean())

746
339020541282.8016


In [330]:
res.shape

(528, 521)

In [331]:
res.flatten().shape

(275088,)

In [332]:
res_gdf.shape

(275088, 5)

In [333]:
for sub_category in sub_category_list:
    print(len(lbr_geo_filt.loc[lbr_geo_filt[sub_category]==1]))
    res = ma.calculate_travel_time(travel_surf, mcp, lbr_geo_filt.loc[lbr_geo_filt[sub_category]==1])[0].copy()
    print(res.mean())
    res_gdf[sub_category] = res.flatten()

746
339020541282.8016
351
339020541289.9395
674
339020541282.9493
437
339020541283.68164
555
339020541283.4872


### also add travel times for health facilities and hospitals for statistics

In [334]:
res_health = ma.calculate_travel_time(travel_surf, mcp, lbr_geo_filt)[0].copy()
hospitals = lbr_geo_filt.loc[lbr_geo_filt.facilitytype=="hospital"].copy()
res_hospital = ma.calculate_travel_time(travel_surf, mcp, hospitals)[0].copy()
res_gdf.loc[:, f"tt_health"] = res_health.flatten()
res_gdf.loc[:, f"tt_hospital"] = res_hospital.flatten()

In [335]:
res_gdf = res_gdf.loc[res_df['pop']>0].copy()
res_gdf = res_gdf.loc[~(res_df['pop'].isna())].copy()

In [336]:
res_gdf

,spatial_index,xy,pop,pointid,geometry,offer_anc,offer_emonc,offer_childimmune,offer_essentialnutrition,offer_adolescentsrhr,tt_health,tt_hospital
736,"(1, 215)",POINT (-9.80417 8.64583),8.980525,736,POINT (-9.80417 8.64583),110.428970,122.193579,110.428970,114.735709,110.428970,110.428970,125.431865
737,"(1, 216)",POINT (-9.79583 8.64583),8.305853,737,POINT (-9.79583 8.64583),111.119326,122.883935,111.119326,115.426065,111.119326,111.119326,126.122221
738,"(1, 217)",POINT (-9.78750 8.64583),13.495267,738,POINT (-9.78750 8.64583),112.452659,124.217268,112.452659,116.759399,112.452659,112.452659,127.455554
739,"(1, 218)",POINT (-9.77917 8.64583),14.163782,739,POINT (-9.77917 8.64583),115.700364,127.464973,115.700364,120.007104,115.700364,115.700364,130.703259
740,"(1, 219)",POINT (-9.77083 8.64583),15.457679,740,POINT (-9.77083 8.64583),111.553164,123.317773,111.553164,115.859904,111.553164,111.553164,126.556059
...,...,...,...,...,...,...,...,...,...,...,...,...
268811,"(515, 496)",POINT (-7.46250 4.36250),3.900976,268811,POINT (-7.46250 4.36250),65.622631,65.622631,65.622631,65.622631,74.345863,65.622631,75.653301
269313,"(516, 477)",POINT (-7.62083 4.35417),10.434936,269313,POINT (-7.62083 4.35417),14.368887,14.368887,14.368887,14.368887,23.092119,14.368887,23.217810
269314,"(516, 478)",POINT (-7.61250 4.35417),75.483665,269314,POINT (-7.61250 4.35417),12.472519,12.472519,12.472519,12.472519,21.195750,12.472519,22.503188
269315,"(516, 479)",POINT (-7.60417 4.35417),70.438751,269315,POINT (-7.60417 4.35417),19.217592,19.217592,19.217592,19.217592,27.940824,19.217592,29.248262


In [350]:
res_gdf_export = res_gdf.drop(['geometry','spatial_index'], axis=1)

In [351]:
res_gdf_export

,xy,pop,pointid,offer_anc,offer_emonc,offer_childimmune,offer_essentialnutrition,offer_adolescentsrhr,tt_health,tt_hospital
736,POINT (-9.80417 8.64583),8.980525,736,110.428970,122.193579,110.428970,114.735709,110.428970,110.428970,125.431865
737,POINT (-9.79583 8.64583),8.305853,737,111.119326,122.883935,111.119326,115.426065,111.119326,111.119326,126.122221
738,POINT (-9.78750 8.64583),13.495267,738,112.452659,124.217268,112.452659,116.759399,112.452659,112.452659,127.455554
739,POINT (-9.77917 8.64583),14.163782,739,115.700364,127.464973,115.700364,120.007104,115.700364,115.700364,130.703259
740,POINT (-9.77083 8.64583),15.457679,740,111.553164,123.317773,111.553164,115.859904,111.553164,111.553164,126.556059
...,...,...,...,...,...,...,...,...,...,...
268811,POINT (-7.46250 4.36250),3.900976,268811,65.622631,65.622631,65.622631,65.622631,74.345863,65.622631,75.653301
269313,POINT (-7.62083 4.35417),10.434936,269313,14.368887,14.368887,14.368887,14.368887,23.092119,14.368887,23.217810
269314,POINT (-7.61250 4.35417),75.483665,269314,12.472519,12.472519,12.472519,12.472519,21.195750,12.472519,22.503188
269315,POINT (-7.60417 4.35417),70.438751,269315,19.217592,19.217592,19.217592,19.217592,27.940824,19.217592,29.248262


In [352]:
# export stats gdf
res_gdf_export.to_file(os.path.join(out_folder, "results_gdf_w_sub_categories.shp"))

/tmp/ipykernel_22074/2710100137.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  res_gdf_export.to_file(os.path.join(out_folder, "results_gdf_w_sub_categories.shp"))


In [353]:
# export tt_health and tt_hospital
aggregator.rasterize_gdf(res_gdf, 'tt_health', raster_path, os.path.join(out_folder, "tt_health_2nd_rd.tif"), nodata=-1)
aggregator.rasterize_gdf(res_gdf, 'tt_hospital', raster_path, os.path.join(out_folder, "tt_hospital_2nd_rd.tif"), nodata=-1)

In [207]:
raster_path = out_pop_surface_std

In [209]:
aggregator.rasterize_gdf(res_gdf, 'offer_anc', raster_path, os.path.join(out_folder, "tt_offer_anc_2nd_rd.tif"), nodata=-1)
aggregator.rasterize_gdf(res_gdf, 'offer_emonc', raster_path, os.path.join(out_folder, "tt_offer_emonc_2nd_rd.tif"), nodata=-1)
aggregator.rasterize_gdf(res_gdf, 'offer_childimmune', raster_path, os.path.join(out_folder, "tt_offer_childimmune_2nd_rd.tif"), nodata=-1)
aggregator.rasterize_gdf(res_gdf, 'offer_essentialnutrition', raster_path, os.path.join(out_folder, "tt_offer_essentialnutrition_2nd_rd.tif"), nodata=-1)
aggregator.rasterize_gdf(res_gdf, 'offer_adolescentsrhr', raster_path, os.path.join(out_folder, "tt_offer_adolescentsrhr_2nd_rd.tif"), nodata=-1)